# Import packages

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import re

import undetected_chromedriver as uc
import importlib
import time

import utils

In [2]:
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
import numpy as np

In [4]:
import concurrent.futures


In [5]:
def reload_utils():
    importlib.reload(utils)

reload_utils()

# Scrape from url df

In [6]:
def calculate_run_time(load_time):
    num = load_time*2664/3600
    print(f"{num} hours")
    return num

In [7]:
url_df = pd.read_excel("data/medifind_providers_v2_1.xlsx")
url_df = url_df.query("url.str.contains('medifind.com')")

FileNotFoundError: [Errno 2] No such file or directory: 'data/medifind_providers_v2_1.xlsx'

In [ ]:
url_df = url_df.drop_duplicates()

In [ ]:
# split url df into chunks of 500 so that web scraping works
n = 500
url_df_parts_top_level = [url_df.iloc[i:i + n] for i in range(0, len(url_df), n)]

all_urls = []
num_chunks = 6
for df in url_df_parts_top_level:
    url_df_parts = np.array_split(df, num_chunks)
    all_urls.append(url_df_parts)

In [ ]:
print(len(all_urls))
for elem in all_urls:
    for x in elem:
        print(x.shape[0])
print([x.shape for x in url_df_parts_top_level])

In [ ]:
# generator_results = []
all_final_dfs = []
for i, url_chunks in enumerate(all_urls):
    # if i == 0:
    #     continue
    print(f"Parallel scraping session: {i+1}")
    with concurrent.futures.ProcessPoolExecutor() as executor:
        generator_results = executor.map(utils.scrape_urls, url_chunks)
        # print("time for next top level chunk and next parallel scraping session...")
    
    # flatten 2d results into 1 list of dictionaries
    final_dict = {k: v for d in generator_results for k, v in d.items()}

    # scrape the html and save it to list
    df_more_details = []
    for url, doc_doctor in final_dict.items():
        doc_doctor = BeautifulSoup(doc_doctor, 'html.parser')
        tmp = utils.scrape_medifind_doctor(doc_doctor)
        tmp["url"] = url
        df_more_details.append(tmp)
    
    # turn list into pandas df
    final_df = pd.DataFrame(df_more_details)
    # write file
    final_df.to_excel(f"data/medifind_providers/medifind_providers_nf_partition_{str(i+1).zfill(3)}.xlsx", index = False)
    all_final_dfs.append(final_df)
    

    # wait 2 minutes between scraping sessions
    # don't wait after finished all sessions
    if i < len(all_urls)-1:
        print("sleeping...")
        
        time.sleep(60*2)

In [ ]:
# flatten 2d results into 1 list of dictionaries
results = [y for x in generator_results for y in x]


In [ ]:
# turn list of dictionaries into 1 dictionary
final_dict = {k: v for d in results for k, v in d.items()}

In [ ]:
df_more_details = []
for url, doc_doctor in final_dict.items():
    doc_doctor = BeautifulSoup(doc_doctor, 'html.parser')
    tmp = utils.scrape_medifind_doctor(doc_doctor)
    tmp["url"] = url
    df_more_details.append(tmp)
    
final_df = pd.DataFrame(df_more_details)

In [ ]:
final_df.shape

In [ ]:
final_df.query("name.isna()")

In [ ]:
final_df.to_excel("data/medifind_providers_data_nf_500.xlsx", index = False)

In [ ]:
combined_df = pd.concat(all_final_dfs)

In [ ]:
combined_df.query("name.isna()")

In [ ]:
combined_df.to_excel("data/medifind_providers/medifind_providers_nf_all.xlsx", index = False)

In [ ]:
"_".join("NF d ds".split(" "))

In [ ]:
conditions = ["Neurofibromatosis", "Neurofibromatosis Type 2 (NF2)", 
                  "Neurofibromatosis Type 3 also known as Schwannomatosis", "Neurofibromatosis Type 6 also known as Cafe-Au-Lait_Spots",
                  "Neurofibromatosis Type 1 (NF1)", "Neurofibromatosis 1-Like Syndrome also known as Legius Syndrome",
                  "Malignant Neurofibroma also known as Malignant Peripheral Nerve Sheath Tumor"]
["Neurofibroma" in x for x in conditions]

In [ ]:
file_name_to_condition_dict = {
    "nf": "Neurofibromatosis",
    "nf1": "Neurofibromatosis Type 1 (NF1)",
    "nf2": "Neurofibromatosis Type 2 (NF2)",
    "nf3": "Neurofibromatosis Type 3 also known as Schwannomatosis",
    "nf6": "Neurofibromatosis Type 6 also known as Cafe-Au-Lait_Spots",
    "legius": "Neurofibromatosis 1-Like Syndrome also known as Legius Syndrome",
    "sheath": "Malignant Neurofibroma also known as Malignant Peripheral Nerve Sheath Tumor",
}

urls_to_scrape_list = []
for k,v in file_name_to_condition_dict.items():
    urls_to_scrape_list.append(pd.read_excel(f"data/medifind_urls/medifind_providers_{k}.xlsx", index_col = 0))

urls_df = pd.concat(urls_to_scrape_list)

    

In [ ]:
urls_df.condition.value_counts()

In [ ]:
set(url_df.url) - set(combined_df.url)

In [ ]:
print(urls_df.shape[0])
url_df.url.nunique()

# Analyze final combined df here

In [ ]:
combined_df.shape[0]

In [ ]:
combined_df.query("name.isna()")

In [ ]:
combined_df.sample(10)

In [ ]:
combined_df.hospitals

In [ ]:
# combined_df.query("'' in hospitals")
combined_df[combined_df["hospitals"].str.contains("Memorial Sl", na=False)]

In [ ]:
def label_city_states(row):
    # try:
    if pd.isna(row["locations"]):
        return ""
            
    location_text = row["locations"] 
    if pd.notna(row["other_locations"]):
        location_text += ";"  + row["other_locations"]
    
    # print(location_text)

    # Define a regular expression pattern to extract city and state pairs
    # pattern = r"\b[A-Z][a-z]+\b, [A-Z]{2}\b"


    # # Use the findall() method to extract all matches of the pattern in the text
    # matches = ";".join(list(set(re.findall(pattern, location_text))))

    # return matches

    matches = re.findall(r"\b([A-Z][a-z]*(?:\s+[A-Z][a-z]*)*),\s*([A-Z]{2})\b", location_text)
    cities_states = ";".join(list(set([f"{match[0]}, {match[1]}" for match in matches])))

    return cities_states

    # except Exception as e:
    #     print(e)
    #     return ""

def label_states(row):
    # try:
    if pd.isna(row["locations"]):
        return ""
            
    location_text = row["locations"] 
    if pd.notna(row["other_locations"]):
        location_text += ";"  + row["other_locations"]

    matches = re.findall(r"\b([A-Z][a-z]*(?:\s+[A-Z][a-z]*)*),\s*([A-Z]{2})\b", location_text)
    states = list(set([f"{match[1]}" for match in matches]))

    return states

combined_df["city_states"] = combined_df.apply(lambda row: label_city_states(row), axis=1)
combined_df["states"] = combined_df.apply(lambda row: label_states(row), axis=1)



In [ ]:
combined_df.query("locations.isna()")

In [ ]:
combined_df.iloc[20].locations

In [ ]:
combined_df.iloc[20].city_states

In [ ]:
combined_df.iloc[274].locations

In [ ]:
for index, s in combined_df.iterrows():
    print(s.locations)

In [ ]:
for index, s in combined_df.iterrows():
    print(s.city_states)

In [ ]:
for index, s in combined_df.iterrows():
    print(index, s.states)

In [ ]:
states_df = combined_df[["name", "states"]]

In [ ]:
states_df["states_backup"] = states_df["states"]

In [ ]:
states_df = states_df.explode('states').fillna('')


In [ ]:
states_df.iloc[18:20]

In [ ]:
states_df.groupby(by="states").agg(num_providers=("name", "count")).reset_index()

In [ ]:
import plotly.express as px


In [ ]:
states_plot_df = states_df.groupby(by="states").agg(num_providers=("name", "count")).reset_index()
fig = px.choropleth(states_plot_df, locations="states", locationmode="USA-states", 
                    color="num_providers", scope="usa", color_continuous_scale = "OrRd",labels={'num_providers': '# Providers'})

# fig.update_layout(
#     coloraxis={"colorscale": [(0, "white"), (0.5, "skyblue"), (1, "blue")]},
# )
fig.update_xaxes(showline=True, linewidth=10, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)

fig.update_layout(title = "<b>US NF Providers</b>",margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
